**Question 1: What is a RESTful API? Explain four core principles of RESTful API design and why they are important.**

**Answer:**
A RESTful API (Representational State Transfer API) is a way for client applications (like web or mobile apps) to communicate with a server over HTTP using a set of simple, consistent rules. It exposes resources (such as users, orders, or products) through URLs and allows clients to perform operations on them using standard HTTP methods like GET, POST, PUT, and DELETE.

Four Core Principles of RESTful API Design
1. Client–Server Separation

The client (frontend) and server (backend) are independent of each other.

The client only handles the user interface.

The server manages data storage and business logic.

Why it’s important:
This separation improves scalability and flexibility. You can change the frontend without affecting the backend and vice versa.

2. Statelessness

Each API request must contain all the information needed to process it. The server does not store client session state between requests.

Why it’s important:
Statelessness makes APIs easier to scale, debug, and cache, because every request is independent and predictable.

3. Resource-Based Design

Everything is treated as a resource, identified by a unique URL.
Example:

/users/10

/orders/45

HTTP methods define actions:

GET → read

POST → create

PUT/PATCH → update

DELETE → remove

Why it’s important:
This keeps APIs clean, intuitive, and consistent, making them easier for developers to understand and use.

4. Use of Standard HTTP Methods and Status Codes

RESTful APIs rely on standard HTTP methods and response codes like:

200 OK

201 Created

400 Bad Request

401 Unauthorized

404 Not Found

500 Internal Server Error

Why it’s important:
Using standards ensures predictable behavior and better interoperability across different systems and clients.

**Question 2: Write Express.js routes for a users resource supporting create, read (all and by id), update, and delete operations.**

**Answer:**
import express from "express";

const router = express.Router();

// Dummy data store
let users = [];
let idCounter = 1;

/* ---------------- CREATE USER ---------------- */
router.post("/users", (req, res) => {
  const { name, email } = req.body;

  const newUser = {
    id: idCounter++,
    name,
    email,
  };

  users.push(newUser);
  res.status(201).json(newUser);
});

/* ---------------- READ ALL USERS ---------------- */
router.get("/users", (req, res) => {
  res.json(users);
});

/* ---------------- READ USER BY ID ---------------- */
router.get("/users/:id", (req, res) => {
  const user = users.find(u => u.id === parseInt(req.params.id));

  if (!user) {
    return res.status(404).json({ message: "User not found" });
  }

  res.json(user);
});

/* ---------------- UPDATE USER ---------------- */
router.put("/users/:id", (req, res) => {
  const user = users.find(u => u.id === parseInt(req.params.id));

  if (!user) {
    return res.status(404).json({ message: "User not found" });
  }

  user.name = req.body.name ?? user.name;
  user.email = req.body.email ?? user.email;

  res.json(user);
});

/* ---------------- DELETE USER ---------------- */
router.delete("/users/:id", (req, res) => {
  const index = users.findIndex(u => u.id === parseInt(req.params.id));

  if (index === -1) {
    return res.status(404).json({ message: "User not found" });
  }

  users.splice(index, 1);
  res.json({ message: "User deleted successfully" });
});

export default router;

**Question 3: What role does Passport.js play in authentication? Outline the login flow using username and password.**

**Answer:**
Passport.js is an authentication middleware for Node.js that simplifies handling authentication in Express applications. It provides a flexible framework to authenticate users using different strategies such as username/password, JWT, OAuth (Google, Facebook, GitHub), etc., without tying authentication logic directly to application code.

Role of Passport.js in Authentication

Acts as a middleware layer between incoming requests and your application logic

Handles credential verification using pluggable strategies

Manages user authentication state (login sessions or tokens)

Keeps authentication logic separate from routes and business code

Why it’s important:
Passport.js makes authentication modular, reusable, and easier to maintain, especially in large applications.

Login Flow Using Username and Password (Local Strategy)
1. Client Sends Login Request

The user submits their credentials (username/email and password) via a POST request.

POST /login
Body: { username, password }
2. Passport Local Strategy Is Triggered

Passport uses the LocalStrategy to intercept the request and extract the username and password.

Compares the username with the database

Verifies the password (usually using hashing like bcrypt)

3. User Verification

If credentials are valid, Passport returns the user object

If credentials are invalid, authentication fails

passport.use(
  new LocalStrategy((username, password, done) => {
    // find user in DB
    // compare hashed password
    return done(null, user);
  })
);
4. Session Handling (Serialize & Deserialize)

For session-based authentication:

serializeUser() stores the user ID in the session

deserializeUser() retrieves the full user object for future requests

passport.serializeUser((user, done) => {
  done(null, user.id);
});

passport.deserializeUser((id, done) => {
  // fetch user by id
  done(null, user);
});
5. Response to Client

On success → login success response (or session cookie)

On failure → error message (401 Unauthorized)

**Question 4: Explain the OAuth login flow using Google in an Express application.**

**Answer:**

In an Express app, Google OAuth login works by redirecting the user to Google for authentication. The user logs in and gives consent. Google then redirects back to the app with an authorization code. Using Passport.js, the server exchanges this code for the user’s profile, creates or finds the user in the database, and establishes a session or issues a token.

**Question 5: Compare session-based authentication and token-based (JWT) authentication.When would you use each?**

**Answer:**
Session-based authentication keeps user state on the server and is suitable for traditional web apps with strong server control. Token-based (JWT) authentication is stateless, stores tokens on the client, and is best for modern APIs, SPAs, and scalable distributed applications.

**Question 6: Write Express middleware that allows access only to authenticated users with role admin.**

**Answer:**
function requireAdmin(req, res, next) {
  // Check if user is authenticated
  if (!req.user) {
    return res.status(401).json({ message: "Unauthorized" });
  }

  // Check if user role is admin
  if (req.user.role !== "admin") {
    return res.status(403).json({ message: "Forbidden: Admin access only" });
  }

  // User is authenticated and admin
  next();
}

export default requireAdmin;

app.delete("/admin/delete-user/:id", requireAdmin, (req, res) => {
  res.json({ message: "User deleted by admin" });
});

**Question 7 : What are SQL/NoSQL injection attacks? Mention one practical prevention technique for Node.js applications.**

**Answer:**

SQL/NoSQL injection attacks occur when an attacker inserts malicious queries into user input to manipulate a database. This can lead to unauthorized data access, data modification, or data deletion.

Prevention technique (Node.js):
Use parameterized queries / prepared statements (or ORM query binding) instead of directly concatenating user input into database queries. This ensures user input is treated as data, not executable code.

**Question 8: What is CORS and why must it be configured in APIs? Write Express.js code to allow requests only from https://client.app.**

**Answer:**
CORS (Cross-Origin Resource Sharing) is a browser security mechanism that controls which domains are allowed to access an API. It must be configured because browsers block requests from different origins by default to prevent security risks like data theft.

Why configure it in APIs?

To allow trusted frontends to access the backend

To block unauthorized or malicious domains

import cors from "cors";
import express from "express";

const app = express();

app.use(
  cors({
    origin: "https://client.app",
  })
);

app.get("/api/data", (req, res) => {
  res.json({ message: "CORS allowed" });
});

app.listen(5000);

**Question 9: What is Next.js and why is it used over plain React? Explain file-based routing using the App Router.**

**Answer:**

Next.js is a React framework that provides features like server-side rendering (SSR), static site generation (SSG), and better performance out of the box. It is used over plain React because it improves SEO, page load speed, and routing without extra configuration.

File-based routing (App Router):
In Next.js, routes are created automatically using the folder structure inside the app/ directory. Each folder represents a URL path, and a page.js file defines the UI for that route. For example, app/blog/page.js maps to /blog. This makes routing simple, organized, and scalable.

**Question 10: Explain dynamic routes, middleware, and one performance feature (Suspense or lazy loading) in Next.js.**

**Answer:**

Dynamic routes:
Dynamic routes in Next.js allow pages to handle variable URL segments using brackets. For example, app/users/[id]/page.js renders pages like /users/1 or /users/5.

Middleware:
Middleware in Next.js runs before a request is completed and is used for tasks like authentication, authorization, redirects, and logging. It executes at the edge and helps control access to routes.

Performance feature (Suspense / lazy loading):
Suspense and lazy loading improve performance by loading components only when needed. This reduces initial page load time and shows a fallback UI while content is loading.

